In [ ]:
import re

In [ ]:
workflows = dict()
parts = []

with open("day19_input.txt") as file:
    for line in file:
        if match := re.match(r"(\w+)\{(.+)\}", line):
            name = match.group(1)
            rules = match.group(2).split(",")
            workflows[name] = rules
        elif match := re.match(r"\{(.+)\}", line):
            parts.append(match.group(1))

In [ ]:
def eval_rules(rules, part):
    for category in part.split(","):
        exec(category)
    for rule in rules:
        *condition, result = rule.split(":")
        if condition and eval(condition[0]):
            break
    return result

# Part 1


In [ ]:
total = 0
for part in parts:
    name = "in"
    while True:
        name = eval_rules(workflows[name], part)
        if name == "R":
            break
        elif name == "A":
            for number in re.findall(r"\d+", part):
                total += int(number)
            break

print("Answer:", total)

# Part 2


In [ ]:
import math

import portion as P

MIN, MAX = 1, 4000

In [ ]:
def string_to_interval(s: str) -> P.Interval:
    """Create an interval from a string."""
    if s.startswith("<"):
        return P.closedopen(MIN, int(s[1:]))
    return P.openclosed(int(s[1:]), MAX)

In [ ]:
def interval_length(interval: P.Interval) -> int:
    """Calculate the length of an interval."""
    return sum(1 for _ in P.iterate(interval, step=1))

In [ ]:
def possible_intervals(
    name: str, intervals: dict[str, P.Interval], solutions: list[dict[str, P.Interval]]
) -> list[dict[str, P.Interval]]:
    if name == "A":
        # We have reached a possible solution
        solutions.append(intervals)
    elif name == "R":
        pass
    else:
        for rule in workflows[name]:
            *condition, result = rule.split(":")
            if condition:
                # Which category and range is the condition about?
                category = condition[0][0]
                category_range = string_to_interval(condition[0][1:])

                # If the condition is satisfied: Branch off to another workflow
                int_copy = intervals.copy()
                int_copy[category] = int_copy[category].intersection(category_range)
                possible_intervals(result, int_copy, solutions)

                # If the condition is NOT satisfied: Check the next rule
                intervals[category] = intervals[category].difference(category_range)
            else:
                # There was no condition, continue to the fallback workflow
                possible_intervals(result, intervals, solutions)

    # Return all possible solutions
    return solutions

In [ ]:
intervals = {
    "x": P.closed(MIN, MAX),
    "m": P.closed(MIN, MAX),
    "a": P.closed(MIN, MAX),
    "s": P.closed(MIN, MAX),
}

solutions = 0
for solution in possible_intervals("in", intervals, []):
    solutions += math.prod(interval_length(interval) for interval in solution.values())

print("Answer:", solutions)